# Train, Validate $\rightarrow$ Train, Test

In this exercise, you will perform empirical comparison of the results of a ten-fold cross validated model with a fully trained model.

## Notes and Guidelines
* Read a dataset from disk and use it for a classification task.
* Construct a Gaussian Naive Bayes classifier and fit it to the phoneme dataset provided.
* Save and re-load a trained classifier.
* Compare K-fold cross-validation scores with the success rate of a fully-trained model.


### Dataset
* Dataset acquired from [KEEL](http://sci2s.ugr.es/keel/dataset.php?cod=105), an excellent resource for finding 'toy' datasets (and a few more serious ones).
    * A description of the dataset is provided at the above link - **read it.**
    * Excerpt: 
    *The aim of this dataset is to distinguish between nasal (class 0) and oral sounds (class 1).
    The class distribution is 3,818 samples in class 0 and 1,586 samples in class 1.
    The phonemes are transcribed as follows: sh as in she, dcl as in dark, iy as the vowel in she, aa as the vowel in dark, and ao as the first vowel in water.*
    
* It is not necessary to fully understand the nature or context of the values in the dataset - only that there are five columns of input (featural) data and one column of output (class) data.

## Handling imports and dataset inclusion

In [1]:
import os
import pandas as pd
import numpy as np

# <import necessary modules> 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
from collections import OrderedDict
import sklearn.model_selection
from sklearn.naive_bayes import GaussianNB

# locate dataset
DATASET = '/dsa/data/all_datasets/phoneme.csv'  # phoneme classification dataset
assert os.path.exists(DATASET)  # check if the file actually exists

## Constructing DataFrame from raw dataset

<span style="background:yellow">**Note**</span>: Variable `dataset` should be used for the dataframe.

In [2]:

dataset = pd.read_csv(DATASET, header=0).sample(frac=1)

# verify dataset shape
print("Dataset shape: ", dataset.shape)

Dataset shape:  (5404, 6)


In [3]:
# show first few lines of the dataset
dataset.head()

,Aa,Ao,Dcl,Iy,Sh,Class
2488,0.381,2.045,-0.306,-0.163,-0.091,0
2066,0.857,2.735,-0.450,-0.187,-0.199,0
5076,0.171,0.958,2.044,-0.762,-0.596,0
1083,0.278,1.134,2.136,0.838,1.005,1
5006,0.682,1.980,1.237,-0.433,-0.517,0


## Splitting data into training and test sets

Split the datasets into training (80%) and testing (20%) sets. 

The below is only necessary if you are interested in visualizing
the data or providing neatly-labeled output within the program.

```python
# extract labels from column headers
phonemes = dataset.columns[0:5].tolist()  # Feature labels
labels = {0: 'Nasal', 1: 'Oral'}  # Class labels
```

In [4]:
# extract features and class data from primary data frame
X = np.array(dataset.iloc[:, :-1])
y = np.array(dataset.Class)

# split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print("Training shapes (X, y): ", X_train.shape, y_train.shape)
print("Testing shapes (X, y): ", X_test.shape, y_test.shape)

Training shapes (X, y):  (4323, 5) (4323,)
Testing shapes (X, y):  (1081, 5) (1081,)


## Constructing the classifier and running automated cross-validation

* Run a 10-fold cross validation with `GaussianNB` classifier
* Print the accuracy scores for these 10 folds

In [5]:
# Your code below this line (Question #E101)
# --------------------------

classifier = GaussianNB()
scores = cross_val_score(classifier, X_train, y_train, cv=10)
print(scores)

[0.75288684 0.73903002 0.79445727 0.7662037  0.75       0.71527778
 0.78240741 0.77777778 0.78935185 0.75925926]


## Training the classifier and pickling to disk
* Learn the model with all the training instances and store to disk

In [6]:
# Your code below this line (Question #E102)
# --------------------------
classifier.fit(X_train, y_train)

import joblib
joblib.dump(classifier, 'GaussianDigits.pkl')

['GaussianDigits.pkl']

## Unpickling the model and making predictions

* Load the saved model 
* Make predictions for the testing set


In [7]:
# Your code below this line (Question #E103)
# --------------------------

# load pickled model
loaded_model = joblib.load('GaussianDigits.pkl')

# make predictions with freshly loaded model
y_pred = classifier.predict(X_test)

# verify input and output shape are appropriate
print("Input vs. output shape:")
print(X_test.shape, y_pred.shape)


Input vs. output shape:
(1081, 5) (1081,)


## Performing final performance comparison

In [8]:
x = (0.75288684 + 0.73903002 + 0.79445727 + 0.7662037 + 0.75 + 0.71527778 +
 0.78240741 + 0.77777778 + 0.78935185 + 0.75925926)
x

7.6266519100000005

In [9]:
# tally up right + wrong 'guesses' by model
true, false = 0, 0
for i, j in zip(y_test, y_pred):
    # print(i, j)
    if i == j:
        true += 1
    else:
        false += 1

# report results numerically and by percentage
true_percent = true / (true + false) * 100
print("Correct guesses: " + str(true) + "\nIncorrect guesses: " + str(false))
print("Percent correct: " + str(true_percent))

# compare to average of cross-validation scores
avg_cv = np.sum(x) / len(scores) * 100
print("Percent cross-validation score (10 folds, average): " + str(avg_cv))

Correct guesses: 820
Incorrect guesses: 261
Percent correct: 75.85568917668826
Percent cross-validation score (10 folds, average): 76.26651910000001


## Measure performance using Scikit Learn modules 

Compute and display the following:
 1. Compute Confusion Matrix
 1. Accuracy
 1. Precision
 1. Recall
 1. $F_1$-Score
 
Add additional cells if required. 

In [15]:
# Your code below this line  (Question #E104)
# --------------------------
from sklearn.metrics import confusion_matrix
# 1.Compute Confusion Matrix
confusion_matrix(y_test, classifier.predict(X_test)) 

array([[574, 190],
       [ 71, 246]])

In [11]:
# 2. Accuracy
from sklearn.dummy import DummyClassifier
dummy_model = DummyClassifier(strategy='most_frequent')
dummy_model.fit(X,y)
print(f"Accuracy: {dummy_model.score(X,y)}")

dummy_model = DummyClassifier(strategy='uniform')
dummy_model.fit(X,y)
print(f"Accuracy: {dummy_model.score(X,y)}")

Accuracy: 0.7065136935603257
Accuracy: 0.5009252405625463


In [12]:
# 3. Percision
from sklearn.metrics import average_precision_score
average_precision_score(y_test, y_pred)

0.5035290273749374

In [13]:
# 4. Recall
from sklearn.metrics import recall_score
recall_score(y_pred, y_test, average="weighted")

0.7585568917668826

In [16]:
# 5. 𝐹1 -Score
from sklearn.metrics import f1_score
f1_score(y_test, np.round(classifier.predict(X_test)).astype('i4'), average='micro')

0.7585568917668825

## Conclusions ?

How did your trained model perform relative to your expectations based on the cross-validation?
Provide your answer in the cell below.

In [ ]:
# Add your answer below this comment  (Question #E105)
# -----------------------------------

The trained model performed slightly worse relative to the expectations based on the cross validation average scores.


# Save your notebook!  Then `File > Close and Halt`